In [ ]:
# works but is weird
!pip install gradio sentence-transformers pandas numpy torch transformers --quiet

import pandas as pd
import numpy as np
import torch
import gradio as gr
import re
from transformers import AutoTokenizer, AutoModel, pipeline
from sentence_transformers import util

# --- Data Loading & Preprocessing ---
df = pd.read_csv('ideas_with_embeddings.csv')
df['Embeddings'] = df['Embeddings'].apply(
    lambda x: np.fromstring(x.strip("[]"), sep=', ')
)

# --- Text Preprocessing ---
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# --- Embedding Generation Setup ---
model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_roberta_embeddings(texts, batch_size=32):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,  # Explicit truncation
            max_length=512,
            return_tensors='pt'
        ).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.extend(batch_embeddings)
    
    return np.array(embeddings)

# --- Similarity Search ---
def find_most_similar(query_embedding, top_k=3):
    similarities = []
    for emb in df['Embeddings']:
        sim = util.pytorch_cos_sim(
            torch.tensor(query_embedding).float(),
            torch.tensor(emb).float()
        ).item()
        similarities.append(sim)
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return df.iloc[top_indices]

# --- Improved Response Generation ---
generator = pipeline(
    'text-generation', 
    model='sshleifer/tiny-gpt2',
    pad_token_id=50256  # Explicitly set pad token
)

def generate_response(query, history):
    # Process query
    processed_query = preprocess_text(query)
    query_embedding = get_roberta_embeddings([processed_query])[0]
    
    # Find similar ideas
    similar_ideas = find_most_similar(query_embedding)
    main_response = "\n".join([f"- {idea}" for idea in similar_ideas['Cleaned_Ideas'].values[:3]])
    
    # Generate additional text with safe limits
    prompt = f"Based on these ideas: {main_response}\nQuestion: {query}\nAnswer:"
    generated_text = generator(
        prompt,
        max_new_tokens=50,  # Generate up to 50 new tokens
        do_sample=True,
        temperature=0.7,
        top_k=40,
        num_return_sequences=1
    )[0]['generated_text'].split("Answer:")[-1].strip()
    
    return f"{main_response}\n\nAdditional Insights:\n{generated_text}"

# --- Gradio Interface ---
demo = gr.ChatInterface(
    generate_response,
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Ask about innovative ideas...", container=False, scale=7),
    title="Idea Generation Assistant",
    examples=["How can we improve urban transportation?", 
             "Suggest sustainable packaging ideas"],
    theme="soft"
)

demo.launch()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/var/folders/49/6ydqkbq172ngzt6p49xfm6b00000gn/T/ipykernel_84775/2723449637.py:98: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and wil

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
